# POST Processing POD Reports

This notebook verifies the uniqueness of Penn holdings identified by the main processing pipeline. It reads from the standardized outputs in `pod-processing-outputs/` and performs final validation using the BorrowDirect API and Selenium-based verification.

## Key Integration Points:
1. **Input**: Reads from pipeline outputs in order of preference:
   - `pod-processing-outputs/statistical_sample_for_api_no_hsp.parquet` (statistical sample)
   - `pod-processing-outputs/physical_books_no_533.parquet` (filtered dataset)
   - `pod-processing-outputs/unique_penn.parquet` (unique Penn records)
   - Legacy Excel fallback support
2. **HSP Filtering**: Already applied in main pipeline (conditionally applied here if needed)
3. **ML Filtering**: Applies machine learning to identify ~1M BorrowDirect-unique records from the 1.6M dataset
4. **BorrowDirect Results**: Leverages existing results or performs fresh API calls with recovery support
5. **HathiTrust Integration**: Checks digital availability for identified holdings
6. **Output**: Saves multiple datasets with confidence intervals:
   - Confirmed Penn-only holdings: `penn_unique_confirmed.xlsx/parquet`
   - Indeterminate holdings: `penn_indeterminate_holdings.xlsx/parquet`
   - ML-filtered BD-unique dataset: `penn_bd_unique_1m_filtered.parquet`
   - Complete verification results and summary statistics

## Enhanced Workflow:
1. **Data Loading & Validation**: Load data from main pipeline outputs with robust column handling and data lineage tracking
2. **Data Processing Overview**: Visual pipeline flow showing all processing stages
3. **HSP Filtering**: Apply only if not already done in main pipeline
4. **API Processing**: Use existing BorrowDirect results or fetch fresh data via API (with sample-based optimization for large datasets)
5. **ML Filtering**: Random Forest model identifies ~1M likely BD-unique records from 1.6M dataset
6. **Selenium Verification**: Sample-based holdings verification (1,000 records) with statistical context
7. **Statistical Extrapolation**: Results extrapolated to full ~1M dataset with 95% confidence intervals
8. **HathiTrust Check**: Digital availability check on representative sample (5,000 records)
9. **Final Export**: Categorized holdings with comprehensive documentation and confidence intervals

## Key Features:
- **Smart Recovery**: Automatically uses existing API results if available
- **Large Dataset Handling**: Uses statistical sampling for datasets >10,000 records
- **Machine Learning**: Random Forest model reduces 1.6M records to ~1M BD-unique holdings
- **Statistical Rigor**: 95% confidence intervals for all extrapolated estimates
- **Memory Management**: Automatic Spark cleanup after ML processing
- **Coverage Monitoring**: Alerts when API coverage is below 50% with actionable suggestions
- **Status Tracking**: Distinguishes between determined, indeterminate, and error states
- **Dual Export**: Tracks both confirmed unique holdings and potentially unique indeterminate records
- **HathiTrust Integration**: Identifies digitization opportunities

## Statistical Methodology:
- **Sampling**: Uses 1,000 record sample for Selenium verification (95% confidence ±3.1%)
- **ML Training**: Trains on sample to identify borrow-direct-unique characteristics
- **Extrapolation**: Projects results to full ~1M ML-filtered dataset with confidence intervals
- **Transparency**: Clear documentation of sample sizes, confidence levels, and margins of error

## Output Interpretation:
The pipeline produces estimates with confidence intervals rather than exact counts:
- **Example**: "~300,000 (287,000-313,000) Penn-unique holdings" instead of just "300,000"
- **Context**: Results show both minimum confirmed and maximum potential unique holdings
- **Coverage**: Automatically monitors and reports API coverage percentage

The pipeline now provides statistically sound estimates with proper confidence intervals, enhanced memory management, and comprehensive quality monitoring throughout the process.

In [1]:
# Load data from main pipeline outputs - Updated and Robust
import pandas as pd
import numpy as np
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size

# Initialize Spark if needed
try:
    spark = SparkSession.getActiveSession()
    if spark is None:
        spark = SparkSession.builder \
            .appName("PostProcessing-Aligned") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
            .getOrCreate()
    print("✅ Spark session ready")
except:
    print("⚠️ Spark not available, using pandas for file reading")
    spark = None

# Replace the input_files list in post-processing.ipynb with:
input_files = [
    "pod-processing-outputs/statistical_sample_for_api_no_hsp.parquet",  # Sample from pod-processing
    "pod-processing-outputs/physical_books_no_533.parquet",  # Final filtered dataset (533 removed)
    "pod-processing-outputs/unique_penn.parquet",  # Basic unique Penn records
    "pod-processing-outputs/penn_overlap_analysis.parquet",  # Alternative analysis file
    "unique_penn_text.xlsx"  # Legacy Excel fallback
]

# Try to load from pipeline outputs
df = None
loaded_from = None

for input_file in input_files:
    if os.path.exists(input_file):
        try:
            print(f"📂 Attempting to load: {input_file}")
            if input_file.endswith('.parquet'):
                if spark:
                    df_spark = spark.read.parquet(input_file)
                    df = df_spark.toPandas()
                else:
                    df = pd.read_parquet(input_file)
                loaded_from = input_file
                print(f"✅ Loaded {len(df):,} records from {input_file}")
                break
            elif input_file.endswith('.xlsx'):
                df = pd.read_excel(input_file)
                loaded_from = input_file
                print(f"✅ Loaded {len(df):,} records from {input_file}")
                break
            elif input_file.endswith('.csv'):
                df = pd.read_csv(input_file)
                loaded_from = input_file
                print(f"✅ Loaded {len(df):,} records from {input_file}")
                break
        except Exception as e:
            print(f"❌ Failed to load {input_file}: {e}")
            continue

if df is None:
    raise FileNotFoundError("❌ No valid input files found. Please run the main pipeline first.")

print(f"\n🎯 Dataset loaded from: {loaded_from}")
print(f"📊 Shape: {df.shape}")
print(f"📋 Columns ({len(df.columns)}): {list(df.columns)}")

# Display basic statistics
print(f"\n📈 Quick Statistics:")
print(f"  Total records: {len(df):,}")
print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/23 09:44:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/23 09:44:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark session ready
📂 Attempting to load: pod-processing-outputs/unique_penn.parquet


✅ Loaded 1,596,684 records from pod-processing-outputs/unique_penn.parquet

🎯 Dataset loaded from: pod-processing-outputs/unique_penn.parquet
📊 Shape: (1596684, 7)
📋 Columns (7): ['F001', 'source', 'match_key', 'id_list', 'is_valid_match_key', 'match_key_message', 'key_array']

📈 Quick Statistics:
  Total records: 1,596,684
  Memory usage: 726.2 MB
  Memory usage: 726.2 MB


# Data Processing Pipeline Overview

This section provides an overview of the complete POD post-processing workflow and data flow.

In [3]:
# Data Processing Pipeline Overview - CORRECTED
print("📊 POD POST-PROCESSING PIPELINE FLOW")
print("="*50)
print("1️⃣ Load data (~1.6M records from unique_penn.parquet)")
print("2️⃣ Apply ML filter → ~1M BD-unique records")
print("3️⃣ Map BorrowDirect IDs (using sample)")
print("4️⃣ Selenium verification (1,000 sample)")
print("5️⃣ Extrapolate results to full ~1M dataset")
print("6️⃣ HathiTrust check (5,000 sample)")
print("7️⃣ Export final estimates")
print("\n📌 Key: Samples are used for API calls due to rate limits")
print("        Results are extrapolated with confidence intervals")

📊 POD POST-PROCESSING PIPELINE FLOW
1️⃣ Load data (~1.6M records from unique_penn.parquet)
2️⃣ Apply ML filter → ~1M BD-unique records
3️⃣ Map BorrowDirect IDs (using sample)
4️⃣ Selenium verification (1,000 sample)
5️⃣ Extrapolate results to full ~1M dataset
6️⃣ HathiTrust check (5,000 sample)
7️⃣ Export final estimates

📌 Key: Samples are used for API calls due to rate limits
        Results are extrapolated with confidence intervals


In [4]:
# Inspect columns and identify key fields - Enhanced with Data Lineage
from datetime import datetime
print("📋 Available columns:")
for i, col in enumerate(df.columns, 1):
    non_null_count = df[col].count()
    null_pct = ((len(df) - non_null_count) / len(df) * 100) if len(df) > 0 else 0
    print(f"  {i:2d}. {col:<30} ({non_null_count:,} non-null, {null_pct:.1f}% null)")

# Enhanced key columns tracking with metadata
key_columns = {
    'record_id': None,
    'match_key': None,
    'borrowdir_results': None,
    'hsp_filtered': False,
    'processing_date': datetime.now().strftime("%Y-%m-%d"),
    'source_file': loaded_from,
    'data_lineage': []
}

# Identify record ID column with validation
for col_name in ['F001', 'record_id', 'mms_id', 'MMSID']:
    if col_name in df.columns:
        key_columns['record_id'] = col_name
        key_columns['data_lineage'].append(f"Using {col_name} as record identifier")
        break

# Identify match key column with validation
for col_name in ['unique_match_key', 'match_key', 'normalized_match_key']:
    if col_name in df.columns:
        key_columns['match_key'] = col_name
        key_columns['data_lineage'].append(f"Using {col_name} for match key comparison")
        break

# Check for existing BorrowDirect results with validation
for col_name in ['borrowdir_ids', 'borrowdir_id', 'borrowdirect_ids', 'borrowdirect_results']:
    if col_name in df.columns:
        key_columns['borrowdir_results'] = col_name
        key_columns['data_lineage'].append(f"Found existing BorrowDirect results in {col_name}")
        break

# Enhanced HSP filtering detection
hsp_status = {
    'filtered': False,
    'source': None,
    'date': None
}

if loaded_from:
    # Check filename for HSP indicators
    if any(term in loaded_from.lower() for term in ['hsp', 'no_hsp', 'filtered']):
        hsp_status['filtered'] = True
        hsp_status['source'] = 'filename'
        key_columns['data_lineage'].append(f"HSP filtering detected from filename: {loaded_from}")

# Check for explicit HSP filtering columns
if 'hsp_filtered' in df.columns:
    hsp_status['filtered'] = True
    hsp_status['source'] = 'column'
    key_columns['data_lineage'].append("HSP filtering verified through column presence")

# Check for HSP filtering timestamp if available
if 'hsp_filtered_date' in df.columns:
    hsp_status['date'] = df['hsp_filtered_date'].iloc[0]
    key_columns['data_lineage'].append(f"HSP filtering date found: {hsp_status['date']}")

key_columns['hsp_filtered'] = hsp_status['filtered']

# Print enhanced status report
print(f"\n=== Data Processing Status ===")
print(f"🔄 Processing Date: {key_columns['processing_date']}")
print(f"📂 Source File: {key_columns['source_file']}")

print(f"\n🔑 Key Columns Status:")
for key, value in key_columns.items():
    if key not in ['processing_date', 'source_file', 'data_lineage']:
        status = "✅" if value else ("⚠️" if key == 'borrowdir_results' else "❌")
        print(f"  {status} {key}: {value}")

print(f"\n📋 Data Lineage:")
for step in key_columns['data_lineage']:
    print(f"  • {step}")

# Enhanced institution analysis
institution_cols = [col for col in df.columns if 'institution' in col.lower() or col in ['POD_organization']]
if institution_cols:
    print(f"\n🏛️ Institution Columns:")
    for col in institution_cols:
        unique_values = df[col].nunique()
        print(f"  • {col} ({unique_values:,} unique values)")

# Enhanced data sample display
print(f"\n📊 Data Sample (first 3 rows):")
display_cols = []
if key_columns['record_id']:
    display_cols.append(key_columns['record_id'])
if key_columns['match_key']:
    display_cols.append(key_columns['match_key'])
if key_columns['borrowdir_results']:
    display_cols.append(key_columns['borrowdir_results'])
if institution_cols:
    display_cols.extend(institution_cols[:1])

# Add key fields for analysis
for field in ['F245', 'F020']:  # Title and ISBN fields
    if field in df.columns:
        display_cols.append(field)

if display_cols:
    print(df[display_cols].head(3))
else:
    print(df.head(3))

# Save processing metadata
processing_metadata = {
    'processing_date': key_columns['processing_date'],
    'source_file': key_columns['source_file'],
    'data_lineage': key_columns['data_lineage'],
    'hsp_status': hsp_status
}

# Store metadata in DataFrame
df.attrs['processing_metadata'] = processing_metadata

📋 Available columns:
   1. F001                           (1,596,684 non-null, 0.0% null)
   2. source                         (1,596,684 non-null, 0.0% null)
   3. match_key                      (1,596,684 non-null, 0.0% null)
   4. id_list                        (0 non-null, 100.0% null)
   5. is_valid_match_key             (1,596,684 non-null, 0.0% null)
   6. match_key_message              (1,596,684 non-null, 0.0% null)
   7. key_array                      (1,596,684 non-null, 0.0% null)

=== Data Processing Status ===
🔄 Processing Date: 2025-07-23
📂 Source File: pod-processing-outputs/unique_penn.parquet

🔑 Key Columns Status:
  ✅ record_id: F001
  ✅ match_key: match_key
  ⚠️ borrowdir_results: None
  ❌ hsp_filtered: False

📋 Data Lineage:
  • Using F001 as record identifier
  • Using match_key for match key comparison

📊 Data Sample (first 3 rows):
               F001                                          match_key
0  9910001563503681  welfare policy for the 1990s edited by p

In [5]:
# Format record ID if needed - Enhanced
if key_columns['record_id']:
    record_col = key_columns['record_id']
    print(f"🔧 Formatting {record_col} column...")
    
    # Store original type for comparison
    original_dtype = df[record_col].dtype
    original_sample = df[record_col].head().tolist()
    
    # Ensure record ID is a string, then apply specific transformations
    df[record_col] = df[record_col].astype(str)
    
    # Replace any occurrence ending with "03680" with "03681" (known data correction)
    corrections_made = df[record_col].str.contains(r'03680$', regex=True, na=False).sum()
    if corrections_made > 0:
        df[record_col] = df[record_col].str.replace(r'03680$', '03681', regex=True)
        print(f"  ✅ Applied {corrections_made} record ID corrections (03680 → 03681)")
    
    # Remove any 'nan' strings that might have been created
    nan_count = (df[record_col] == 'nan').sum()
    if nan_count > 0:
        df[record_col] = df[record_col].replace('nan', pd.NA)
        print(f"  ✅ Cleaned {nan_count} 'nan' string values")
    
    print(f"  Original dtype: {original_dtype}")
    print(f"  New dtype: {df[record_col].dtype}")
    print(f"  Sample original values: {original_sample}")
    print(f"  Sample formatted values: {df[record_col].head().tolist()}")
    
    # Check for any remaining issues
    null_count = df[record_col].isnull().sum()
    if null_count > 0:
        print(f"  ⚠️ Warning: {null_count} null values in record ID column")
else:
    print("⚠️ No record ID column found - skipping record ID formatting")
    print("Available columns:", list(df.columns))

🔧 Formatting F001 column...
  Original dtype: object
  New dtype: object
  Sample original values: ['9910001563503681', '9910004073503681', '9910004943503681', '9910006103503681', '9910008193503681']
  Sample formatted values: ['9910001563503681', '9910004073503681', '9910004943503681', '9910006103503681', '9910008193503681']
  Original dtype: object
  New dtype: object
  Sample original values: ['9910001563503681', '9910004073503681', '9910004943503681', '9910006103503681', '9910008193503681']
  Sample formatted values: ['9910001563503681', '9910004073503681', '9910004943503681', '9910006103503681', '9910008193503681']


In [6]:
# Check match key uniqueness and completeness
if key_columns['match_key']:
    match_col = key_columns['match_key']
    print(f"🔍 Analyzing {match_col} column...")
    
    # Basic statistics
    total_records = len(df)
    unique_keys = df[match_col].nunique()
    is_unique = df[match_col].is_unique
    
    print(f"  📊 Basic Statistics:")
    print(f"    Total records: {total_records:,}")
    print(f"    Unique match keys: {unique_keys:,}")
    print(f"    All keys unique: {is_unique}")
    if not is_unique:
        duplicates = total_records - unique_keys
        print(f"    Duplicate records: {duplicates:,} ({duplicates/total_records*100:.1f}%)")
    
    # Check for null/empty values
    null_count = df[match_col].isnull().sum()
    empty_count = (df[match_col] == '').sum() if df[match_col].dtype == 'object' else 0
    total_missing = null_count + empty_count
    
    print(f"  🚫 Missing Data:")
    print(f"    Null values: {null_count:,}")
    print(f"    Empty strings: {empty_count:,}")
    print(f"    Total missing: {total_missing:,} ({total_missing/total_records*100:.1f}%)")
    
    # Analyze match key patterns
    if df[match_col].dtype == 'object' and total_missing < total_records:
        valid_keys = df[match_col].dropna()
        valid_keys = valid_keys[valid_keys != '']
        
        if len(valid_keys) > 0:
            key_lengths = valid_keys.str.len()
            print(f"  📏 Key Length Analysis:")
            print(f"    Min length: {key_lengths.min()}")
            print(f"    Max length: {key_lengths.max()}")
            print(f"    Median length: {key_lengths.median()}")
            
            # Show sample keys of different lengths
            print(f"  🔤 Sample keys:")
            for i, key in enumerate(valid_keys.head(5)):
                print(f"    {i+1}. {key[:50]}{'...' if len(key) > 50 else ''} (len: {len(key)})")
    
    # If there are missing match keys, show sample records
    if total_missing > 0:
        print(f"\n⚠️ Records with missing match keys:")
        missing_sample = df[df[match_col].isnull() | (df[match_col] == '')]
        display_cols = [key_columns['record_id'], match_col]
        display_cols = [col for col in display_cols if col is not None]
        
        # Add some additional identifying columns if available
        for col in ['F245', 'title', 'F020', 'isbn', 'POD_organization']:
            if col in df.columns:
                display_cols.append(col)
                break
        
        print(missing_sample[display_cols].head())
        
        # Option to filter out missing keys
        print(f"\n❓ Should we filter out records with missing match keys? ({total_missing:,} would be removed)")
else:
    print("❌ No match key column found - cannot proceed with BorrowDirect verification")
    print("This is required for API calls. Please check the main pipeline outputs.")

🔍 Analyzing match_key column...
  📊 Basic Statistics:
    Total records: 1,596,684
    Unique match keys: 1,473,174
    All keys unique: False
    Duplicate records: 123,510 (7.7%)
  🚫 Missing Data:
    Null values: 0
    Empty strings: 0
    Total missing: 0 (0.0%)
  📊 Basic Statistics:
    Total records: 1,596,684
    Unique match keys: 1,473,174
    All keys unique: False
    Duplicate records: 123,510 (7.7%)
  🚫 Missing Data:
    Null values: 0
    Empty strings: 0
    Total missing: 0 (0.0%)
  📏 Key Length Analysis:
    Min length: 4
    Max length: 5210
    Median length: 81.0
  🔤 Sample keys:
    1. welfare policy for the 1990s edited by phoebe h co... (len: 80)
    2. rural labourers in bengal 1880 to 1980 willem van ... (len: 85)
    3. earthquake hazards and the design of constructed f... (len: 134)
    4. diario del primo amore giacomo leopardi introduzio... (len: 74)
    5. american law and the constitutional order historic... (len: 135)
  📏 Key Length Analysis:
    Min len

In [7]:
# HSP filtering - Conditional and Enhanced
initial_count = len(df)

if key_columns['hsp_filtered']:
    print("✅ HSP filtering already applied in main pipeline - skipping")
    print(f"   Current record count: {initial_count:,}")
    
elif os.path.exists('hsp/hsp-removed-mmsid.txt'):
    print("🔧 Applying HSP filtering from hsp/hsp-removed-mmsid.txt...")
    
    # Load HSP MMSIDs to remove
    with open('hsp/hsp-removed-mmsid.txt') as f:
        hsp_removed_mmsid = [line.strip() for line in f.readlines() if line.strip()]
    
    print(f"   Loaded {len(hsp_removed_mmsid):,} HSP MMSIDs to remove")
    
    # Get the record ID column
    record_col = key_columns['record_id']
    if record_col:
        # Apply HSP filtering: Remove rows with MMSIDs that are in hsp_removed_mmsid
        before_count = len(df)
        
        # Convert both to strings for comparison
        df[record_col] = df[record_col].astype(str)
        hsp_set = set(str(mmsid) for mmsid in hsp_removed_mmsid)
        
        # CORRECT: Remove rows with MMSIDs that are in hsp_removed_mmsid (use ~ for NOT)
        mask = ~df[record_col].isin(hsp_set)
        df = df[mask].copy()
        
        after_count = len(df)
        removed_count = before_count - after_count
        
        print(f"✅ HSP filtering complete:")
        print(f"   Records before: {before_count:,}")
        print(f"   Records after: {after_count:,}")
        print(f"   Records removed: {removed_count:,} ({removed_count/before_count*100:.1f}%)")
        
        if removed_count == 0:
            print("   ℹ️ No records were removed - HSP MMSIDs may not be present in this dataset")
    else:
        print("❌ Cannot apply HSP filtering - no record ID column found")
        
elif os.path.exists('hsp-removed-mmsid.txt'):
    print("🔧 Applying HSP filtering from current directory...")
    
    with open('hsp-removed-mmsid.txt') as f:
        hsp_removed_mmsid = [line.strip() for line in f.readlines() if line.strip()]
    
    record_col = key_columns['record_id'] 
    if record_col:
        before_count = len(df)
        df[record_col] = df[record_col].astype(str)
        hsp_set = set(str(mmsid) for mmsid in hsp_removed_mmsid)
        mask = ~df[record_col].isin(hsp_set)  # CORRECT: Added ~ operator
        df = df[mask].copy()
        
        after_count = len(df)
        removed_count = before_count - after_count
        
        print(f"✅ HSP filtering complete: removed {removed_count:,} records")
    else:
        print("❌ Cannot apply HSP filtering - no record ID column found")
        
else:
    print("⚠️ HSP filtering file not found - proceeding without HSP filtering")
    print("   This may be acceptable if HSP filtering was already applied in the main pipeline")

print(f"\n📊 Current dataset size: {len(df):,} records")

⚠️ HSP filtering file not found - proceeding without HSP filtering
   This may be acceptable if HSP filtering was already applied in the main pipeline

📊 Current dataset size: 1,596,684 records


In [8]:
# Data Validation and Match Key Preparation - Fixed
import pandas as pd
import os
from datetime import datetime
import json

print("="*60)
print("DATA VALIDATION AND PREPARATION")
print("="*60)

# Create output directory if it doesn't exist
os.makedirs('pod-processing-outputs', exist_ok=True)

# Validate that we have the necessary data
validation_status = {
    'has_data': 'df' in locals() and df is not None,
    'has_key_columns': 'key_columns' in locals(),
    'match_key_found': False,
    'match_key_valid': False,
    'ready_for_api': False
}

if not validation_status['has_data']:
    print("❌ No dataframe found. Please run the data loading cells first.")
else:
    print(f"✅ Dataframe loaded: {len(df):,} records")
    
    # Check for match key
    if validation_status['has_key_columns'] and key_columns.get('match_key'):
        match_col = key_columns['match_key']
        validation_status['match_key_found'] = True
        print(f"✅ Match key column found: {match_col}")
        
        # Validate match key data
        total_records = len(df)
        valid_keys = df[match_col].notna() & (df[match_col] != '')
        valid_count = valid_keys.sum()
        
        # Convert numpy types to Python types
        validation_status['match_key_valid'] = bool(valid_count > 0)
        
        print(f"\n📊 Match Key Statistics:")
        print(f"  Total records: {total_records:,}")
        print(f"  Valid match keys: {valid_count:,}")
        print(f"  Missing/empty keys: {total_records - valid_count:,}")
        print(f"  Percentage valid: {valid_count/total_records*100:.1f}%")
        
        if valid_count == 0:
            print("\n❌ No valid match keys found - cannot proceed with API calls")
        else:
            validation_status['ready_for_api'] = True
            print(f"\n✅ Ready for BorrowDirect API calls with {valid_count:,} records")
            
            # Save validation status with proper type conversion
            validation_file = "pod-processing-outputs/data_validation_status.json"
            
            # Convert all values to JSON-serializable types
            validation_data = {
                'timestamp': datetime.now().isoformat(),
                'valid_record_count': int(valid_count),  # Convert numpy int to Python int
                'has_data': bool(validation_status['has_data']),
                'has_key_columns': bool(validation_status['has_key_columns']),
                'match_key_found': bool(validation_status['match_key_found']),
                'match_key_valid': bool(validation_status['match_key_valid']),
                'ready_for_api': bool(validation_status['ready_for_api']),
                'match_key_column': match_col,
                'total_records': int(total_records)
            }
            
            with open(validation_file, 'w') as f:
                json.dump(validation_data, f, indent=2)
            print(f"💾 Validation status saved to: {validation_file}")
    else:
        print("❌ No match key column found")
        print("   Cannot proceed with BorrowDirect verification")
        
        # Try to join with penn unique file if available
        penn_unique_files = [
            "pod-processing-outputs/unique_penn_corrected.parquet",
            "pod-processing-outputs/unique_penn.parquet",
            "unique_penn_corrected.xlsx"
        ]
        
        print("\n🔍 Looking for Penn unique files with match keys...")
        for file in penn_unique_files:
            if os.path.exists(file):
                print(f"   Found: {file}")
                # Suggest joining in next step
                validation_status['suggested_join_file'] = file
                break
        else:
            print("   No Penn unique files found")

# Print final status
print("\n" + "="*40)
print("VALIDATION SUMMARY")
print("="*40)
for key, value in validation_status.items():
    if isinstance(value, bool):
        status_icon = "✅" if value else "❌"
    else:
        status_icon = "ℹ️"
    print(f"{status_icon} {key}: {value}")

# Store validation results for next cell
if validation_status['ready_for_api']:
    print("\n✅ Proceed to next cell for BorrowDirect API fetching")
else:
    print("\n⚠️ Data issues need to be resolved before API calls")

DATA VALIDATION AND PREPARATION
✅ Dataframe loaded: 1,596,684 records
✅ Match key column found: match_key

📊 Match Key Statistics:
  Total records: 1,596,684
  Valid match keys: 1,596,684
  Missing/empty keys: 0
  Percentage valid: 100.0%

✅ Ready for BorrowDirect API calls with 1,596,684 records
💾 Validation status saved to: pod-processing-outputs/data_validation_status.json

VALIDATION SUMMARY
✅ has_data: True
✅ has_key_columns: True
✅ match_key_found: True
✅ match_key_valid: True
✅ ready_for_api: True

✅ Proceed to next cell for BorrowDirect API fetching


In [9]:
# Clear any existing execution state and restart cleanly
import os
import shutil

# Clean up all guard files and start fresh
guard_files = [
    "pod-processing-outputs/borrowdirect_api_guard.txt",
    "pod-processing-outputs/borrowdirect_api_complete.json",
    "pod-processing-outputs/api_checkpoint.json"
]

for file in guard_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"✅ Removed {file}")

print("\n🔧 All guard files cleared. Ready for fresh API fetch.")
print("📝 Run the next cell to start the API fetch process.")


🔧 All guard files cleared. Ready for fresh API fetch.
📝 Run the next cell to start the API fetch process.


In [10]:
# Enhanced BorrowDirect API Fetch with Sample Detection and Recovery Support
import time
import requests
import pandas as pd
import os
import json
import ast  # For safely evaluating string representations of lists
from typing import List
from urllib.parse import quote
from datetime import datetime

# Check if we should proceed
if not os.path.exists("pod-processing-outputs/data_validation_status.json"):
    raise ValueError("❌ Please run the validation cell first")

# Check if already completed
if os.path.exists("pod-processing-outputs/borrowdirect_api_complete.json"):
    print("✅ API fetch already completed. Check the results.")
    with open("pod-processing-outputs/borrowdirect_api_complete.json", 'r') as f:
        info = json.load(f)
    print(f"   Completed at: {info.get('completed_at')}")
    print(f"   Records processed: {info.get('records_processed', 0):,}")
else:
    # RECOVERY APPROACH: Check for existing sample results
    api_sample_path = 'pod-processing-outputs/api_sample_results.csv'
    if os.path.exists(api_sample_path) and key_columns['match_key']:
        print("\n" + "="*60)
        print("BORROWDIRECT API RECOVERY - USING EXISTING RESULTS")
        print("="*60 + "\n")
        
        print(f"📂 Loading API sample results from: {api_sample_path}")
        api_sample_df = pd.read_csv(api_sample_path)
        print(f"   ✅ Loaded {len(api_sample_df):,} API sample results")
        
        match_col = key_columns['match_key']
        
        # Convert string representations of lists back to actual lists
        if 'borrowdir_ids' in api_sample_df.columns:
            api_sample_df['borrowdir_ids'] = api_sample_df['borrowdir_ids'].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else 
                        ([] if pd.isna(x) else x)
            )
            
            # Create mapping from match keys to borrowdir_ids
            match_key_to_ids = dict(zip(api_sample_df[match_col], api_sample_df['borrowdir_ids']))
            
            # Apply to your full dataset
            print("   🔄 Applying saved API results to full dataset...")
            df['borrowdir_ids'] = df[match_col].map(match_key_to_ids).fillna('').apply(
                lambda x: x if isinstance(x, list) else []
            )
            
            # Update tracking
            key_columns['borrowdir_results'] = 'borrowdir_ids'
            
            # Count records with results
            has_results = df['borrowdir_ids'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False).sum()
            print(f"   ✅ Applied API results to full dataset of {len(df):,} records")
            print(f"   Records with results: {has_results:,} ({has_results/len(df)*100:.1f}%)")
            
            # ADDED: Coverage check
            coverage = has_results / len(df) * 100
            if coverage < 50:
                print(f"   ⚠️ Low API coverage ({coverage:.1f}%) - consider:")
                print("      • Expanding API sample size")
                print("      • Checking match key quality")
                print("      • Verifying BorrowDirect service availability")
            
            # Save completion marker
            with open("pod-processing-outputs/borrowdirect_api_complete.json", 'w') as f:
                json.dump({
                    'completed_at': datetime.now().isoformat(),
                    'recovery': True,
                    'records_processed': len(api_sample_df),
                    'records_with_results': int(has_results),
                    'recovery_file': api_sample_path
                }, f, indent=2)
            
            print(f"\n✅ Recovery complete! Used existing API results from {api_sample_path}")
    else:
        # ORIGINAL APPROACH: No existing results, perform API calls
        print("\n" + "="*60)
        print("BORROWDIRECT API FETCH - STARTING")
        print("="*60 + "\n")
        
        # Define API function
        def get_borrowdir_ids(match_key: str) -> List[str]:
            if pd.isna(match_key) or match_key == '':
                return []
            
            try:
                encoded_key = quote(match_key, safe='')
                url = f"https://borrowdirect.reshare.indexdata.com/api/v1/search?lookfor={encoded_key}"
                response = requests.get(url, timeout=30, headers={'User-Agent': 'POD-Processing/1.0'})
                response.raise_for_status()
                data = response.json()
                ids = list(set(record['id'] for record in data.get('records', [])))
                time.sleep(1.5)  # Rate limiting
                return ids
            except Exception as e:
                print(f"Error fetching {match_key[:20]}...: {str(e)}")
                return []
        
        # Check for existing results
        if 'borrowdir_ids' in df.columns:
            print("✅ BorrowDirect results already exist in dataframe")
            has_results = df['borrowdir_ids'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False).sum()
            print(f"   Records with results: {has_results:,}")
            df_api = df  # Use existing dataframe
        else:
            # LARGE DATASET HANDLING - Check if we should use a sample
            if len(df) > 10000:
                print(f"⚠️ LARGE DATASET DETECTED: {len(df):,} records")
                print(f"   Full processing would take approximately {len(df) * 1.5 / 60 / 24:.1f} days")
                print(f"   Switching to statistical sample for API validation...")
                
                # Look for sample files in different formats (ONLY UNZIPPED)
                sample_files = [
                    "pod-processing-outputs/statistical_sample_for_api_no_hsp.parquet",
                    "pod-processing-outputs/statistical_sample_for_api_no_hsp.csv"
                ]
                
                # Try to load regular files
                sample_loaded = False
                for sample_file in sample_files:
                    if os.path.exists(sample_file):
                        print(f"   Found sample file: {sample_file}")
                        try:
                            if sample_file.endswith('.parquet'):
                                df_api = pd.read_parquet(sample_file)
                            elif sample_file.endswith('.csv'):
                                # Check if it's a file or directory (Spark CSV output)
                                if os.path.isdir(sample_file):
                                    print(f"   Detected Spark CSV directory: {sample_file}")
                                    # Find CSV part files in the directory
                                    part_files = [f for f in os.listdir(sample_file) 
                                                if f.startswith('part-') and f.endswith('.csv')]
                                    
                                    if part_files:
                                        # Read the first part file
                                        first_part = os.path.join(sample_file, part_files[0])
                                        print(f"   Reading part file: {first_part}")
                                        df_api = pd.read_csv(first_part)
                                        
                                        # If there are multiple part files, append them
                                        if len(part_files) > 1:
                                            print(f"   Found {len(part_files)} part files, combining...")
                                            dfs = [df_api]
                                            for part in part_files[1:]:
                                                part_path = os.path.join(sample_file, part)
                                                dfs.append(pd.read_csv(part_path))
                                            df_api = pd.concat(dfs, ignore_index=True)
                                    else:
                                        # Try to find any CSV file in the directory
                                        csv_files = [f for f in os.listdir(sample_file) if f.endswith('.csv')]
                                        if csv_files:
                                            first_csv = os.path.join(sample_file, csv_files[0])
                                            print(f"   Reading CSV file: {first_csv}")
                                            df_api = pd.read_csv(first_csv)
                                        else:
                                            raise FileNotFoundError(f"No CSV files found in directory {sample_file}")
                                else:
                                    # Regular CSV file
                                    df_api = pd.read_csv(sample_file)
                            
                            print(f"   ✅ Loaded {len(df_api):,} records from sample")
                            sample_loaded = True
                            break
                        except Exception as e:
                            print(f"   ❌ Failed to load {sample_file}: {e}")
                
                # If no sample loaded, create one (SKIPPING ZIPPED FILES)
                if not sample_loaded:
                    print("   ⚠️ No sample file found - creating a random sample")
                    sample_size = min(1000, int(len(df) * 0.01))  # 1% or max 1000 records
                    df_api = df.sample(n=sample_size, random_state=42)
                    print(f"   ✅ Created random sample with {len(df_api):,} records")
                    
                    # Save for future use
                    sample_output = "pod-processing-outputs/generated_api_sample.csv" 
                    df_api.to_csv(sample_output, index=False)
                    print(f"   💾 Saved sample to {sample_output} for future use")
            else:
                # Dataset is small enough to process directly
                df_api = df
            
            # Process data
            match_col = key_columns['match_key']
            valid_df = df_api[df_api[match_col].notna() & (df_api[match_col] != '')].copy()
            
            print(f"\n📊 Processing {len(valid_df):,} records for API validation...")
            print(f"⏱️  Estimated time: {len(valid_df) * 1.5 / 60:.1f} minutes\n")
            
            # Process with simple progress tracking
            results = []
            start_time = time.time()
            
            # Save checkpoint every 100 records for potential recovery
            checkpoint_interval = 100
            checkpoint_file = "pod-processing-outputs/api_checkpoint.json"
            
            for i, (idx, row) in enumerate(valid_df.iterrows()):
                result = get_borrowdir_ids(row[match_col])
                results.append(result)
                
                # Save checkpoint at intervals
                if (i + 1) % checkpoint_interval == 0:
                    # Save current results to CSV for potential recovery
                    checkpoint_df = valid_df.iloc[:i+1].copy()
                    checkpoint_df['borrowdir_ids'] = results
                    checkpoint_df.to_csv(api_sample_path, index=False)
                    print(f"   💾 Checkpoint saved at {i+1}/{len(valid_df)} records")
                
                # Progress update every 50 records
                if (i + 1) % 50 == 0 or i == len(valid_df) - 1:
                    elapsed = time.time() - start_time
                    rate = (i + 1) / elapsed if elapsed > 0 else 0
                    eta = (len(valid_df) - (i + 1)) / rate if rate > 0 else 0
                    print(f"Progress: {i+1}/{len(valid_df)} ({(i+1)/len(valid_df)*100:.1f}%) - ETA: {eta/60:.1f} min")
            
            # Apply results to the sample
            valid_df['borrowdir_ids'] = results
            
            # Save final sample results for potential future recovery
            valid_df.to_csv(api_sample_path, index=False)
            print(f"   💾 Saved complete API results to {api_sample_path}")
            
            # If we're using a sample, we need to merge differently
            if df_api is not df:
                print("\n🔄 Processing sample results...")
                # Save sample results for reference
                valid_df.to_parquet("pod-processing-outputs/api_sample_results.parquet", index=False)


BORROWDIRECT API RECOVERY - USING EXISTING RESULTS

📂 Loading API sample results from: pod-processing-outputs/api_sample_results.csv
   ✅ Loaded 1,000 API sample results
   🔄 Applying saved API results to full dataset...
   ✅ Applied API results to full dataset of 1,596,684 records
   Records with results: 15,021 (0.9%)
   ⚠️ Low API coverage (0.9%) - consider:
      • Expanding API sample size
      • Checking match key quality
      • Verifying BorrowDirect service availability

✅ Recovery complete! Used existing API results from pod-processing-outputs/api_sample_results.csv
   ✅ Applied API results to full dataset of 1,596,684 records
   Records with results: 15,021 (0.9%)
   ⚠️ Low API coverage (0.9%) - consider:
      • Expanding API sample size
      • Checking match key quality
      • Verifying BorrowDirect service availability

✅ Recovery complete! Used existing API results from pod-processing-outputs/api_sample_results.csv


In [10]:
# Try specifying an older stable version
!pip install pyarrow==9.0.0

# Or try installing with binary wheels only (no compilation)
!pip install --only-binary :all: pyarrow

# If you have Homebrew, you might need to install Arrow first
# !brew install apache-arrow
# !pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 21.3 MB/s eta 0:00:0000:0100:01


In [11]:
# Save intermediate results to standardized output directory
import os
import ast  # For safely evaluating string representations of lists

# Ensure output directory exists
os.makedirs('pod-processing-outputs', exist_ok=True)

# ADDED: Check if we should load and apply saved API results
api_sample_path = 'pod-processing-outputs/api_sample_results.csv'
if os.path.exists(api_sample_path) and key_columns['match_key']:
    print(f"📂 Loading API sample results from: {api_sample_path}")
    api_sample_df = pd.read_csv(api_sample_path)
    print(f"   ✅ Loaded {len(api_sample_df):,} API sample results")
    
    match_col = key_columns['match_key']
    
    # Convert string representations of lists back to actual lists
    if 'borrowdir_ids' in api_sample_df.columns:
        api_sample_df['borrowdir_ids'] = api_sample_df['borrowdir_ids'].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else 
                    ([] if pd.isna(x) else x)
        )
        
        # Create mapping from match keys to borrowdir_ids
        match_key_to_ids = dict(zip(api_sample_df[match_col], api_sample_df['borrowdir_ids']))
        
        # Apply to your full dataset
        print("   🔄 Applying saved API results to full dataset...")
        df['borrowdir_ids'] = df[match_col].map(match_key_to_ids).fillna('').apply(
            lambda x: x if isinstance(x, list) else []
        )
        
        # Update tracking
        key_columns['borrowdir_results'] = 'borrowdir_ids'
        
        # Count records with results
        has_results = df['borrowdir_ids'].apply(lambda x: len(x) > 0 if isinstance(x, list) else False).sum()
        print(f"   ✅ Applied API results to full dataset of {len(df):,} records")
        print(f"   Records with results: {has_results:,} ({has_results/len(df)*100:.1f}%)")

# Save current state with BorrowDirect results
output_file = 'pod-processing-outputs/post-processing-with-borrowdir_ids.csv'
df.to_csv(output_file, index=False)
print(f"✅ Saved {len(df):,} records to {output_file}")

# Try to save as Parquet for better performance
parquet_file = 'pod-processing-outputs/post-processing-with-borrowdir_ids.parquet'
try:
    df.to_parquet(parquet_file, index=False)
    parquet_saved = True
    print(f"✅ Saved {len(df):,} records to {parquet_file}")
except ImportError:
    print(f"⚠️ Could not save to Parquet format - missing pyarrow or fastparquet package")
    print(f"   Install with: pip install pyarrow")
    parquet_saved = False

# Display summary statistics
print(f"\n📊 Saved Dataset Summary:")
print(f"   Records: {len(df):,}")
print(f"   Columns: {len(df.columns)}")
print(f"   File size (CSV): {os.path.getsize(output_file) / 1024**2:.1f} MB")
if parquet_saved and os.path.exists(parquet_file):
    print(f"   File size (Parquet): {os.path.getsize(parquet_file) / 1024**2:.1f} MB")

📂 Loading API sample results from: pod-processing-outputs/api_sample_results.csv
   ✅ Loaded 1,000 API sample results
   🔄 Applying saved API results to full dataset...
   ✅ Applied API results to full dataset of 1,596,684 records
   Records with results: 15,021 (0.9%)
   ✅ Applied API results to full dataset of 1,596,684 records
   Records with results: 15,021 (0.9%)
✅ Saved 1,596,684 records to pod-processing-outputs/post-processing-with-borrowdir_ids.csv
✅ Saved 1,596,684 records to pod-processing-outputs/post-processing-with-borrowdir_ids.csv
✅ Saved 1,596,684 records to pod-processing-outputs/post-processing-with-borrowdir_ids.parquet

📊 Saved Dataset Summary:
   Records: 1,596,684
   Columns: 8
   File size (CSV): 405.1 MB
   File size (Parquet): 217.2 MB
✅ Saved 1,596,684 records to pod-processing-outputs/post-processing-with-borrowdir_ids.parquet

📊 Saved Dataset Summary:
   Records: 1,596,684
   Columns: 8
   File size (CSV): 405.1 MB
   File size (Parquet): 217.2 MB


In [12]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 18.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2
ERROR: pip's dependency resolver does not currently take into account

In [ ]:
import time
import math
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def check_up_holdings_selenium(borrowdir_id: str, debug: bool = False) -> tuple:
    """
    Check if holdings are exclusive to University of Pennsylvania using Selenium.
    Returns a tuple of (status, is_penn_only) where:
    - status: 'determined', 'indeterminate', or 'error'
    - is_penn_only: True if only Penn holds the item, False otherwise
    """
    # Skip if borrowdir_id is None, NaN, or empty
    if (borrowdir_id is None or 
        (isinstance(borrowdir_id, float) and math.isnan(borrowdir_id)) or
        borrowdir_id == '' or borrowdir_id == 'nan'):
        if debug:
            print("Skipping due to empty/invalid borrowdir_id")
        return ('error', False)

    url = f"https://borrowdirect.reshare.indexdata.com/Record/{borrowdir_id}/Holdings"

    # Configure Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    
    driver = None
    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.set_page_load_timeout(30)
        
        if debug:
            print(f"Accessing URL: {url}")
        
        driver.get(url)
        
        # Wait for dynamic content to load
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.tab-content")))
        
        # Check for "not available" message first
        not_available_message = "This item is not available through BorrowDirect"
        page_text = driver.page_source
        
        if not_available_message in page_text:
            if debug:
                print(f"⚠️ Item not available through BorrowDirect - status indeterminate")
            return ('indeterminate', False)
        
        # Locate the main tab content container
        tab_content = driver.find_element(By.CSS_SELECTOR, "div.tab-content")
        
        # Within tab_content, get the active holdings pane
        holdings_div = tab_content.find_element(By.CSS_SELECTOR, "div.tab-pane.holdings-tab.active")
        
        # Look for h3 elements within the holdings pane
        h3_tags = holdings_div.find_elements(By.TAG_NAME, "h3")
        institutions = set(tag.text.strip() for tag in h3_tags if tag.text.strip())
        
        if debug:
            print(f"Institutions found: {institutions}")
        
        # Check if only University of Pennsylvania holds the item
        is_penn_only = (institutions == {"University of Pennsylvania"})
        
        if debug and is_penn_only:
            print("✅ Penn-only holding confirmed")
        elif debug:
            print(f"❌ Multiple institutions: {institutions}")
            
        return ('determined', is_penn_only)
        
    except Exception as e:
        if debug:
            print(f"Error encountered: {e}")
        return ('error', False)
    finally:
        if driver:
            driver.quit()

# Example debugging usage
print("Testing with known ID...")
status, result = check_up_holdings_selenium("c7e30f8c-83e0-467e-aa74-f6f525e10e72", debug=True)
print(f"Status: {status}, Penn-only: {result}")

# Prepare data for holdings verification
if key_columns['borrowdir_results']:
    borrowdir_col = key_columns['borrowdir_results']
    
    print(f"\n🔍 Preparing holdings verification using {borrowdir_col} column...")
    
    # Create a copy for processing
    verification_df = df.copy()
    
    # Explode borrowdir_ids if they are in list format
    if verification_df[borrowdir_col].apply(lambda x: isinstance(x, list)).any():
        print("   📊 Exploding list-format BorrowDirect IDs...")
        verification_df = verification_df.explode(borrowdir_col).reset_index(drop=True)
        print(f"   Expanded to {len(verification_df):,} records for verification")
    
    # Filter out empty/invalid BorrowDirect IDs
    valid_mask = (
        verification_df[borrowdir_col].notna() & 
        (verification_df[borrowdir_col] != '') & 
        (verification_df[borrowdir_col] != 'nan')
    )

    if verification_df[borrowdir_col].dtype == 'object':
        # First convert to string, then apply the NOT operation to the boolean result
        is_list_str = verification_df[borrowdir_col].astype(str).str.startswith('[')
        valid_mask = valid_mask & (~is_list_str)
    
    verification_df = verification_df[valid_mask].copy()
    
    print(f"   📊 Records ready for verification: {len(verification_df):,}")
    
    # ADDED: Take a sample if the dataset is too large
    if len(verification_df) > 1000:
        print(f"   ⚠️ Dataset too large ({len(verification_df):,} records) - taking a 1,000 record sample")
        
        # Calculate confidence interval for 1000 sample
        total_records = len(verification_df)
        sample_size = 1000
        margin_of_error = 1.96 * math.sqrt(0.25 / sample_size) * 100  # Conservative 50% assumption
        
        print(f"\\nSample Parameters for Selenium Verification:")
        print(f"📊 Sample size: {sample_size:,} records")
        print(f"📊 Population: {total_records:,} records") 
        print(f"📊 Sampling rate: {sample_size/total_records*100:.1f}%")
        print(f"📊 Statistical confidence: 95% ±{margin_of_error:.1f}% (assuming random sampling)")
        print(f"📊 This sample represents the {total_records:,} Penn-unique holdings that need verification")
        print(f"🔍 Testing BorrowDirect availability via web interface automation\\n")
        print(f"Results will be extrapolated to estimate availability across all {total_records:,} records\\n")
        
        verification_df = verification_df.sample(n=1000, random_state=42)
        print(f"   ✅ Sample created with {len(verification_df):,} records")
    
    if len(verification_df) > 0:
        # Sample a few records for testing first
        sample_size = min(5, len(verification_df))
        sample_df = verification_df.head(sample_size).copy()
        
        print(f"\n   🧪 Testing with {sample_size} sample records first...")
        
        # Test with debug output - now handling tuples
        sample_results = sample_df[borrowdir_col].apply(
            lambda x: check_up_holdings_selenium(x, debug=True)
        )
        sample_df['status'] = sample_results.apply(lambda x: x[0])
        sample_df['up_holdings'] = sample_results.apply(lambda x: x[1])
        
        # Show results with status breakdown
        penn_only_count = sample_df['up_holdings'].sum()
        determined_count = (sample_df['status'] == 'determined').sum()
        indeterminate_count = (sample_df['status'] == 'indeterminate').sum()
        error_count = (sample_df['status'] == 'error').sum()
        
        print(f"\n   📊 Sample results:")
        print(f"      Penn-only holdings: {penn_only_count}/{sample_size}")
        print(f"      Determined: {determined_count}")
        print(f"      Indeterminate: {indeterminate_count}")
        print(f"      Errors: {error_count}")
        
        if penn_only_count > 0 or indeterminate_count > 0:
            print(f"\n   ✅ Found verifiable records - proceeding with full verification")
            
            # Apply to full dataset (without debug for speed)
            print(f"   🔄 Verifying all {len(verification_df):,} records...")
            
            # Process in batches to show progress
            batch_size = 100
            results = []
            
            for i in range(0, len(verification_df), batch_size):
                batch_end = min(i + batch_size, len(verification_df))
                batch_df = verification_df.iloc[i:batch_end]
                
                batch_results = batch_df[borrowdir_col].apply(
                    lambda x: check_up_holdings_selenium(x, debug=False)
                )
                results.extend(batch_results)
                
                # Progress update
                if i > 0:
                    print(f"      Progress: {batch_end}/{len(verification_df)} ({batch_end/len(verification_df)*100:.1f}%)")
            
            # Apply results
            verification_df['status'] = [r[0] for r in results]
            verification_df['up_holdings'] = [r[1] for r in results]
            
            # Summary statistics
            total_penn_only = verification_df['up_holdings'].sum()
            total_determined = (verification_df['status'] == 'determined').sum()
            total_indeterminate = (verification_df['status'] == 'indeterminate').sum()
            total_errors = (verification_df['status'] == 'error').sum()
            
            print(f"\n   ✅ Holdings verification complete!")
            print(f"     Total verified records: {len(verification_df):,}")
            print(f"     Status breakdown:")
            print(f"       - Determined: {total_determined:,} ({total_determined/len(verification_df)*100:.1f}%)")
            print(f"       - Indeterminate: {total_indeterminate:,} ({total_indeterminate/len(verification_df)*100:.1f}%)")
            print(f"       - Errors: {total_errors:,} ({total_errors/len(verification_df)*100:.1f}%)")
            
            # FIXED: Safe division for Penn-only percentage
            if total_determined > 0:
                penn_percentage = total_penn_only/total_determined*100
                print(f"     Penn-only holdings: {total_penn_only:,} ({penn_percentage:.1f}% of determined)")
            else:
                print(f"     Penn-only holdings: {total_penn_only:,} (no determined records to calculate percentage)")
            
            # Option to include indeterminate records
            if total_indeterminate > 0:
                print(f"\n   ℹ️ Note: {total_indeterminate:,} records are indeterminate (not available through BorrowDirect)")
                print(f"      These may still be unique to Penn but cannot be verified through this system")
            
        else:
            print(f"   ⚠️ No Penn-only holdings found in sample - check BorrowDirect data quality")
            verification_df['status'] = 'error'
            verification_df['up_holdings'] = False
    else:
        print("   ❌ No valid BorrowDirect IDs found for verification")
        verification_df['status'] = 'error'
        verification_df['up_holdings'] = False
        
else:
    print("❌ No BorrowDirect results column found - cannot perform holdings verification")
    verification_df = df.copy()
    verification_df['status'] = 'error'
    verification_df['up_holdings'] = False

Testing with known ID...
Accessing URL: https://borrowdirect.reshare.indexdata.com/Record/c7e30f8c-83e0-467e-aa74-f6f525e10e72/Holdings
Accessing URL: https://borrowdirect.reshare.indexdata.com/Record/c7e30f8c-83e0-467e-aa74-f6f525e10e72/Holdings
⚠️ Item not available through BorrowDirect - status indeterminate
Status: indeterminate, Penn-only: False

🔍 Preparing holdings verification using borrowdir_ids column...
⚠️ Item not available through BorrowDirect - status indeterminate
Status: indeterminate, Penn-only: False

🔍 Preparing holdings verification using borrowdir_ids column...
   📊 Exploding list-format BorrowDirect IDs...
   📊 Exploding list-format BorrowDirect IDs...
   Expanded to 1,870,545 records for verification
   Expanded to 1,870,545 records for verification
   📊 Records ready for verification: 288,882
   ⚠️ Dataset too large (288,882 records) - taking a 1,000 record sample
\nSample Parameters for Selenium Verification:
📊 Sample size: 1,000 records
📊 Population: 288,882 r

In [ ]:
# Save the verification results
import pandas as pd
import os

# Ensure output directory exists
os.makedirs('pod-processing-outputs', exist_ok=True)

# Check if verification_df exists and has the verification columns
if 'verification_df' in locals() and 'status' in verification_df.columns and 'up_holdings' in verification_df.columns:
    # Save the full verification results
    verification_output = "pod-processing-outputs/selenium_verification_results.parquet"
    verification_df.to_parquet(verification_output, index=False)
    print(f"✅ Saved {len(verification_df):,} verification results to {verification_output}")
    
    # Also save as CSV for easier viewing
    csv_output = "pod-processing-outputs/selenium_verification_results.csv"
    verification_df.to_csv(csv_output, index=False)
    print(f"✅ Saved verification results to {csv_output}")
    
    # Display summary of what was saved
    print(f"\n📊 Verification Results Summary:")
    print(f"   Total records verified: {len(verification_df):,}")
    print(f"   Columns saved: {list(verification_df.columns)}")
    
    # Status breakdown
    status_counts = verification_df['status'].value_counts()
    print(f"\n   Status Breakdown:")
    for status, count in status_counts.items():
        print(f"   - {status}: {count:,} ({count/len(verification_df)*100:.1f}%)")
    
    # Penn-only findings
    penn_only_count = verification_df['up_holdings'].sum()
    print(f"\n   Penn-only holdings found: {penn_only_count:,}")
    
    # Sample of results
    print(f"\n📋 Sample of verification results:")
    display_cols = ['status', 'up_holdings']
    if 'borrowdir_ids' in verification_df.columns:
        display_cols.insert(0, 'borrowdir_ids')
    if key_columns.get('match_key') and key_columns['match_key'] in verification_df.columns:
        display_cols.insert(0, key_columns['match_key'])
    
    print(verification_df[display_cols].head(10))
    
else:
    print("❌ No verification results found to save")
    print("Please run the Selenium verification cell first")

In [ ]:
# NEW CELL: Spark ML Filtering to Identify ~1M BD-Unique Records
print("\n" + "="*60)
print("SPARK ML FILTERING - IDENTIFYING ~1M BD-UNIQUE RECORDS")
print("="*60 + "\n")

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, regexp_extract, length, count, avg, sum
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
import json

# Ensure Spark session is active
if 'spark' not in locals() or spark is None:
    spark = SparkSession.builder \
        .appName("BD-Unique-ML-Filter") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()

# Load the full 1.6M dataset
print("📂 Loading full Penn unique dataset...")
full_df_spark = spark.read.parquet("pod-processing-outputs/unique_penn.parquet")
full_count = full_df_spark.count()
print(f"   Loaded {full_count:,} records")

# Load verification results as training data
print("\n📂 Loading verification results for training...")
train_df = spark.read.parquet("pod-processing-outputs/selenium_verification_results.parquet")

# Create binary label for BD-unique (confirmed or indeterminate)
train_df = train_df.withColumn(
    "is_bd_unique",
    when(
        (col("status") == "indeterminate") | 
        (col("up_holdings") == True), 
        1
    ).otherwise(0)
)

bd_unique_count = train_df.filter(col("is_bd_unique") == 1).count()
print(f"   Training on {train_df.count()} verified records")
print(f"   BD-unique in training set: {bd_unique_count} ({bd_unique_count/train_df.count()*100:.1f}%)")

# Feature engineering function
def create_bd_features(df):
    """Create features that predict BorrowDirect uniqueness"""
    
    # Extract publication year
    df = df.withColumn(
        "pub_year",
        regexp_extract(col("F260"), r"(\d{4})", 1).cast("int")
    )
    
    # Age-based features
    df = df.withColumn(
        "is_pre_1900", when(col("pub_year") < 1900, 1).otherwise(0)
    ).withColumn(
        "is_pre_1950", when(col("pub_year") < 1950, 1).otherwise(0)
    ).withColumn(
        "is_post_2000", when(col("pub_year") >= 2000, 1).otherwise(0)
    )
    
    # Material type indicators
    df = df.withColumn(
        "is_special_material",
        when(
            col("F245").rlike("(?i)(manuscript|papers|collection|archive|thesis|dissertation)") |
            col("F300").rlike("(?i)(microform|photograph|slides|manuscript)"),
            1
        ).otherwise(0)
    )
    
    # Local content
    df = df.withColumn(
        "is_local_content",
        when(
            col("F245").rlike("(?i)(Philadelphia|Pennsylvania|Penn)") |
            col("F260").rlike("(?i)(Philadelphia|Pennsylvania)"),
            1
        ).otherwise(0)
    )
    
    # Publisher features
    df = df.withColumn(
        "is_university_press",
        when(col("F260").rlike("(?i)university"), 1).otherwise(0)
    ).withColumn(
        "no_standard_publisher",
        when(col("F260").rlike("(?i)(s\\.n\\.|sine nomine)"), 1).otherwise(0)
    )
    
    # Format features
    df = df.withColumn(
        "no_isbn",
        when(col("F020").isNull() | (col("F020") == ""), 1).otherwise(0)
    ).withColumn(
        "title_length",
        length(col("F245"))
    )
    
    return df

# Apply features to both datasets
print("\n🔧 Engineering features...")
train_df = create_bd_features(train_df)
full_df_spark = create_bd_features(full_df_spark)

# Build ML pipeline
feature_cols = [
    "is_pre_1900", "is_pre_1950", "is_post_2000",
    "is_special_material", "is_local_content",
    "is_university_press", "no_standard_publisher",
    "no_isbn", "title_length"
]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="is_bd_unique",
    numTrees=100,
    maxDepth=10,
    seed=42
)

pipeline = Pipeline(stages=[assembler, rf])

# Train model
print("\n🎯 Training Random Forest model...")
model = pipeline.fit(train_df)

# Apply to full dataset
print("\n📊 Applying model to full dataset...")
predictions = model.transform(full_df_spark)

# Extract probability of being BD-unique
predictions = predictions.withColumn(
    "bd_unique_probability",
    col("probability").getItem(1)
)

# Check initial predictions
initial_bd_unique = predictions.filter(col("prediction") == 1).count()
print(f"\nInitial predictions: {initial_bd_unique:,} BD-unique records")

# Fine-tune threshold to get ~1M records
if initial_bd_unique > 1100000:
    threshold = 0.6
    print(f"Adjusting threshold to {threshold} to reduce count...")
elif initial_bd_unique < 900000:
    threshold = 0.4
    print(f"Adjusting threshold to {threshold} to increase count...")
else:
    threshold = 0.5

# Apply threshold
bd_unique_filtered = predictions.filter(col("bd_unique_probability") > threshold)
final_count = bd_unique_filtered.count()

print(f"\n✅ Final BD-unique count: {final_count:,}")
print(f"   Reduction: {full_count:,} → {final_count:,} ({(full_count-final_count)/full_count*100:.1f}% filtered out)")

# Analyze composition
print("\n📊 Composition of BD-unique set:")
composition = bd_unique_filtered.agg(
    count("*").alias("total"),
    sum("is_pre_1900").alias("pre_1900"),
    sum("is_special_material").alias("special_materials"),
    sum("is_local_content").alias("local_content"),
    avg("bd_unique_probability").alias("avg_confidence")
).collect()[0]

print(f"  Pre-1900: {composition['pre_1900']:,} ({composition['pre_1900']/composition['total']*100:.1f}%)")
print(f"  Special materials: {composition['special_materials']:,} ({composition['special_materials']/composition['total']*100:.1f}%)")
print(f"  Local content: {composition['local_content']:,} ({composition['local_content']/composition['total']*100:.1f}%)")
print(f"  Average confidence: {composition['avg_confidence']:.3f}")

# Save filtered dataset
output_path = "pod-processing-outputs/penn_bd_unique_1m_filtered.parquet"
bd_unique_filtered.write.mode("overwrite").parquet(output_path)
print(f"\n💾 Saved {final_count:,} BD-unique records to {output_path}")

# Convert to pandas for downstream processing
print("\n🔄 Converting to pandas for final processing...")
df_bd_unique_pandas = bd_unique_filtered.toPandas()

# Update the main df to use the filtered dataset
df = df_bd_unique_pandas
print(f"✅ Updated main dataframe with {len(df):,} BD-unique records")

# Save summary
ml_summary = {
    "original_records": int(full_count),
    "bd_unique_filtered": int(final_count),
    "reduction_pct": float((full_count - final_count) / full_count * 100),
    "threshold_used": float(threshold),
    "composition": {
        "pre_1900": int(composition['pre_1900']),
        "special_materials": int(composition['special_materials']),
        "local_content": int(composition['local_content']),
        "avg_confidence": float(composition['avg_confidence'])
    }
}

with open("pod-processing-outputs/bd_ml_filtering_summary.json", "w") as f:
    json.dump(ml_summary, f, indent=2)

print("\n✅ ML filtering complete! Proceed to final export step.")

# ADDED: Memory cleanup after ML filtering
print("\n🧹 Cleaning up Spark memory...")
spark.catalog.clearCache()
spark.stop()
spark = None
print("✅ Spark session terminated and memory released")

In [ ]:
# Save ML-filtered dataset as checkpoint
print("\n💾 Saving ML-filtered dataset checkpoint...")
ml_filtered_checkpoint = "pod-processing-outputs/ml_filtered_checkpoint.parquet"
df.to_parquet(ml_filtered_checkpoint, index=False)
print(f"✅ Checkpoint saved: {ml_filtered_checkpoint}")

# Also save as CSV for inspection
csv_checkpoint = "pod-processing-outputs/ml_filtered_checkpoint.csv"
df.to_csv(csv_checkpoint, index=False)
print(f"✅ CSV checkpoint saved: {csv_checkpoint}")

In [ ]:
import pandas as pd
import os
import json

# Ensure output directory exists
os.makedirs('pod-processing-outputs', exist_ok=True)

# Check if verification_df exists and has the required columns
if 'verification_df' not in locals() or 'up_holdings' not in verification_df.columns:
    print("⚠️ Verification results not found in memory")
    print("🔄 Attempting to reload from saved files...")
    
    if os.path.exists("pod-processing-outputs/selenium_verification_results.parquet"):
        verification_df = pd.read_parquet("pod-processing-outputs/selenium_verification_results.parquet")
        print(f"✅ Reloaded {len(verification_df):,} verification results")
    else:
        print("❌ No verification results file found")
        raise ValueError("Cannot proceed without verification results")

# Filter records based on verification results
if 'up_holdings' in verification_df.columns and 'status' in verification_df.columns:
    # Get confirmed Penn-only holdings (determined as Penn-only)
    df_penn_only = verification_df[verification_df['up_holdings'] == True].copy()
    
    # Get indeterminate records for separate tracking
    df_indeterminate = verification_df[verification_df['status'] == 'indeterminate'].copy()
    
    # Remove duplicates based on match key or record ID
    if key_columns['match_key']:
        initial_count = len(df_penn_only)
        df_penn_only = df_penn_only.drop_duplicates(subset=[key_columns['match_key']])
        dedup_count = initial_count - len(df_penn_only)
        if dedup_count > 0:
            print(f"🔧 Removed {dedup_count:,} duplicate records based on match key")
            
        if len(df_indeterminate) > 0:
            initial_indet = len(df_indeterminate)
            df_indeterminate = df_indeterminate.drop_duplicates(subset=[key_columns['match_key']])
            indet_dedup = initial_indet - len(df_indeterminate)
            if indet_dedup > 0:
                print(f"🔧 Removed {indet_dedup:,} duplicate indeterminate records")
                
    elif key_columns['record_id']:
        initial_count = len(df_penn_only)
        df_penn_only = df_penn_only.drop_duplicates(subset=[key_columns['record_id']])
        dedup_count = initial_count - len(df_penn_only)
        if dedup_count > 0:
            print(f"🔧 Removed {dedup_count:,} duplicate records based on record ID")
            
        if len(df_indeterminate) > 0:
            initial_indet = len(df_indeterminate)
            df_indeterminate = df_indeterminate.drop_duplicates(subset=[key_columns['record_id']])
            indet_dedup = initial_indet - len(df_indeterminate)
            if indet_dedup > 0:
                print(f"🔧 Removed {indet_dedup:,} duplicate indeterminate records")
    
    # Calculate status breakdown
    total_determined = (verification_df['status'] == 'determined').sum()
    total_indeterminate = len(df_indeterminate)
    total_errors = (verification_df['status'] == 'error').sum()
    
    # NEW: Check if ML filtering was applied
    ml_filtered = os.path.exists("pod-processing-outputs/penn_bd_unique_1m_filtered.parquet")
    if ml_filtered:
        # Load ML summary for accurate counts
        if os.path.exists("pod-processing-outputs/bd_ml_filtering_summary.json"):
            with open("pod-processing-outputs/bd_ml_filtering_summary.json", 'r') as f:
                ml_summary = json.load(f)
            original_dataset_size = ml_summary['original_records']
            ml_filtered_size = ml_summary['bd_unique_filtered']
        else:
            original_dataset_size = 1600000  # Approximate
            ml_filtered_size = len(df) if 'df' in locals() else 1000000
    else:
        original_dataset_size = len(df) if 'df' in locals() else len(verification_df)
        ml_filtered_size = original_dataset_size
    
    # Calculate sample rate based on verification vs ML-filtered dataset
    sample_rate = len(verification_df) / ml_filtered_size if ml_filtered_size > 0 else 1
    is_sample = len(verification_df) < ml_filtered_size
    
    print(f"\n📊 Final Results Summary:")
    if ml_filtered:
        print(f"   Original dataset: {original_dataset_size:,} records")
        print(f"   ML-filtered to: {ml_filtered_size:,} BD-unique records")
    print(f"   Verification sample: {len(verification_df):,} records ({sample_rate*100:.1f}% of {'ML-filtered' if ml_filtered else 'total'})")
    
    print(f"\n   Status Breakdown (from verification sample):")
    print(f"   - Determined: {total_determined:,} ({total_determined/len(verification_df)*100:.1f}%)")
    print(f"   - Indeterminate: {total_indeterminate:,} ({total_indeterminate/len(verification_df)*100:.1f}%)")
    print(f"   - Errors: {total_errors:,} ({total_errors/len(verification_df)*100:.1f}%)")
    
    # Enhanced reporting with ML context
    if is_sample:
        # Calculate estimations for ML-filtered dataset
        est_confirmed_min = int(ml_filtered_size * len(df_penn_only) / len(verification_df)) if len(verification_df) > 0 else 0
        est_indeterminate = int(ml_filtered_size * len(df_indeterminate) / len(verification_df)) if len(verification_df) > 0 else 0
        est_potential_max = est_confirmed_min + est_indeterminate
        
        print(f"\n   📈 Estimated BD-Unique Penn Holdings (extrapolated to ML-filtered ~1M):")
        print(f"   Confirmed minimum: ~{est_confirmed_min:,} holdings")
        print(f"   Indeterminate (possibly unique): ~{est_indeterminate:,} holdings") 
        print(f"   Potential maximum: ~{est_potential_max:,} holdings")
        print(f"   (if all indeterminate records are also unique to Penn)")
        
        # ADDED: Confidence intervals for extrapolated estimates
        # Using 95% confidence interval for 1,000 sample
        import math
        margin_of_error = 1.96 * math.sqrt(0.25 / len(verification_df)) * ml_filtered_size
        est_confirmed_low = max(0, est_confirmed_min - margin_of_error)
        est_confirmed_high = est_confirmed_min + margin_of_error
        
        print(f"   📊 95% Confidence Interval for confirmed: {est_confirmed_low:,.0f} - {est_confirmed_high:,.0f} holdings")
        
        # Show verification sample breakdown
        print(f"\n   📊 From verification sample:")
        print(f"   - Confirmed Penn-only: {len(df_penn_only):,} ({len(df_penn_only)/len(verification_df)*100:.1f}% of sample)")
        print(f"   - Indeterminate: {len(df_indeterminate):,} ({len(df_indeterminate)/len(verification_df)*100:.1f}% of sample)")
        if total_determined > 0:
            print(f"   - Of determined records, {len(df_penn_only)/total_determined*100:.1f}% were Penn-only")
    
    # Export indeterminate records separately if they exist
    if len(df_indeterminate) > 0:
        indet_excel = "pod-processing-outputs/penn_indeterminate_holdings.xlsx"
        indet_parquet = "pod-processing-outputs/penn_indeterminate_holdings.parquet"
        df_indeterminate.to_excel(indet_excel, index=False)
        df_indeterminate.to_parquet(indet_parquet, index=False)
        print(f"\n📝 Exported {len(df_indeterminate):,} indeterminate records:")
        print(f"   - {indet_excel}")
        print(f"   - {indet_parquet}")
        print(f"   ℹ️ These may be unique to Penn but cannot be verified through BorrowDirect")
    
    if len(df_penn_only) > 0:
        # Export confirmed Penn-only holdings
        excel_output = "pod-processing-outputs/penn_unique_confirmed.xlsx"
        df_penn_only.to_excel(excel_output, index=False)
        print(f"\n✅ Exported {len(df_penn_only):,} confirmed Penn-only records to {excel_output}")
        
        # Also save as Parquet
        parquet_output = "pod-processing-outputs/penn_unique_confirmed.parquet"
        df_penn_only.to_parquet(parquet_output, index=False)
        print(f"✅ Exported {len(df_penn_only):,} Penn-only records to {parquet_output}")
        
        # Save detailed verification results
        verification_output = "pod-processing-outputs/holdings_verification_results.parquet"
        verification_df.to_parquet(verification_output, index=False)
        print(f"✅ Saved full verification results to {verification_output}")
        
        # Display sample of Penn-only records
        print(f"\n📋 Sample Penn-only holdings:")
        display_cols = []
        if key_columns['record_id']:
            display_cols.append(key_columns['record_id'])
        if key_columns['match_key']:
            display_cols.append(key_columns['match_key'])
        
        # Add title/isbn columns if available
        for col in ['F245', 'title', 'F020', 'isbn']:
            if col in df_penn_only.columns:
                display_cols.append(col)
                break
        
        print(df_penn_only[display_cols].head() if display_cols else df_penn_only.head())
        
        # Enhanced summary statistics with ML context
        summary = {
            'processing_pipeline': {
                'original_dataset': original_dataset_size,
                'ml_filtered': ml_filtered,
                'ml_filtered_size': ml_filtered_size,
                'verification_sample_size': len(verification_df)
            },
            'verification_results': {
                'sample_rate': round(sample_rate * 100, 2),
                'status_breakdown': {
                    'determined': int(total_determined),
                    'indeterminate': int(total_indeterminate),
                    'errors': int(total_errors)
                },
                'penn_only_confirmed': len(df_penn_only),
                'penn_only_pct_of_determined': round(len(df_penn_only)/total_determined*100, 2) if total_determined > 0 else 0,
                'indeterminate_records': len(df_indeterminate)
            },
            'extrapolation': {
                'target_dataset': 'ML-filtered ~1M BD-unique records' if ml_filtered else 'Full dataset',
                'estimated_penn_only_min': est_confirmed_min if is_sample else len(df_penn_only),
                'estimated_indeterminate': est_indeterminate if is_sample else len(df_indeterminate),
                'estimated_penn_only_max': est_potential_max if is_sample else (len(df_penn_only) + len(df_indeterminate)),
                'is_extrapolated': is_sample
            },
            'output_files': {
                'confirmed': [excel_output, parquet_output],
                'indeterminate': [indet_excel, indet_parquet] if len(df_indeterminate) > 0 else [],
                'verification_results': verification_output
            },
            'input_file': loaded_from if 'loaded_from' in locals() else 'unknown'
        }
        
        # Save summary
        summary_output = "pod-processing-outputs/final_verification_summary.json"
        with open(summary_output, 'w') as f:
            json.dump(summary, f, indent=2)
        print(f"\n✅ Saved processing summary to {summary_output}")
        
        # Final interpretation note
        print(f"\n📌 FINAL INTERPRETATION:")
        if ml_filtered and is_sample:
            print(f"   From the original {original_dataset_size:,} Penn records:")
            print(f"   • ML identified ~{ml_filtered_size:,} as likely BD-unique")
            print(f"   • Verification sample suggests ~{est_confirmed_min:,}-{est_potential_max:,} are Penn-only")
            print(f"   • This represents {est_confirmed_min/original_dataset_size*100:.1f}%-{est_potential_max/original_dataset_size*100:.1f}% of the original dataset")
        elif is_sample:
            print(f"   These results are based on a {sample_rate*100:.1f}% sample.")
            print(f"   The full dataset likely contains ~{est_confirmed_min:,}-{est_potential_max:,} Penn-only holdings.")
        else:
            print(f"   Verified {len(verification_df):,} records directly.")
            print(f"   Found {len(df_penn_only):,} confirmed Penn-only holdings.")
        
    else:
        print("\n⚠️ No confirmed Penn-only holdings found in the verification sample")
        if len(df_indeterminate) > 0:
            print(f"   However, {len(df_indeterminate):,} indeterminate records were exported for review")
        
else:
    print("❌ No holdings verification was performed - cannot create Penn-only export")
    print("Please ensure the holdings verification step completed successfully.")

# HathiTrust Digital Availability Check


In [ ]:
# Check which unique Penn holdings are already digitized in HathiTrust
print("\n" + "="*60)
print("HATHITRUST DIGITAL AVAILABILITY CHECK")
print("="*60)

# Import required modules
import sys
import os
import pandas as pd

# Add HathiTrust directory to path
sys.path.append('hathitrust')

try:
    from hathitrust_availability_checker_excel import HathiTrustFullScanner
    
    # UPDATED: Check ML-filtered dataset if available
    datasets_to_check = []
    
    # First priority: Check the ML-filtered ~1M dataset
    if os.path.exists("pod-processing-outputs/ml_filtered_checkpoint.parquet"):
        print("📂 Loading ML-filtered ~1M BD-unique dataset for HathiTrust check...")
        df_ml_filtered = pd.read_parquet("pod-processing-outputs/ml_filtered_checkpoint.parquet")
        datasets_to_check.append(('ml_filtered_1m', df_ml_filtered, 'ML-filtered BD-unique (~1M)'))
        print(f"   ✅ Loaded {len(df_ml_filtered):,} ML-filtered records")
        
        # For large dataset, use sampling approach
        if len(df_ml_filtered) > 10000:
            print(f"   ⚠️ Large dataset ({len(df_ml_filtered):,} records) - will check a representative sample")
            sample_size = min(5000, int(len(df_ml_filtered) * 0.005))  # 0.5% or max 5000
            df_sample = df_ml_filtered.sample(n=sample_size, random_state=42)
            datasets_to_check = [('ml_filtered_sample', df_sample, f'ML-filtered sample ({sample_size:,} of {len(df_ml_filtered):,})')]
            
            # Save sample info for reporting
            sample_info = {
                'full_dataset_size': len(df_ml_filtered),
                'sample_size': sample_size,
                'sample_rate': sample_size / len(df_ml_filtered)
            }
    else:
        # Fallback to verification results if ML-filtered not available
        print("⚠️ ML-filtered dataset not found, checking verification results instead...")
        
        if 'df_penn_only' in locals() and len(df_penn_only) > 0:
            datasets_to_check.append(('df_penn_only', df_penn_only, 'Penn-only confirmed'))
        
        if 'df_indeterminate' in locals() and len(df_indeterminate) > 0:
            datasets_to_check.append(('df_indeterminate', df_indeterminate, 'indeterminate'))
    
    if not datasets_to_check:
        print("❌ No holdings found to check")
        print("Please ensure the ML filtering or holdings verification step completed successfully")
    else:
        for df_name, df_to_check, description in datasets_to_check:
            print(f"\nChecking {len(df_to_check):,} {description} holdings for HathiTrust availability...")
            
            # Save temporary Excel file with proper column names
            temp_file = f'pod-processing-outputs/temp_hathitrust_input_{df_name}.xlsx'
            
            # FIXED: Use key_columns['record_id'] instead of hardcoded 'F001'
            record_id_col = key_columns['record_id'] if 'key_columns' in locals() and key_columns.get('record_id') else 'F001'
            
            # Ensure borrowdir_col is defined
            borrowdir_col = key_columns.get('borrowdir_results', 'borrowdir_ids') if 'key_columns' in locals() else 'borrowdir_ids'
            
            # Prepare columns for HathiTrust checker
            hathi_df = pd.DataFrame({
                'MMS_ID': df_to_check[record_id_col] if record_id_col in df_to_check.columns else df_to_check.index,
                'F245': df_to_check['F245'] if 'F245' in df_to_check.columns else '',
                'F020_str': df_to_check['F020'].astype(str) if 'F020' in df_to_check.columns else '',
                'F010_str': df_to_check['F010'].astype(str) if 'F010' in df_to_check.columns else '',
                'F260_str': df_to_check['F260'].astype(str) if 'F260' in df_to_check.columns else '',
                'id_list_str': df_to_check['F035'].astype(str) if 'F035' in df_to_check.columns else '',
                'borrowdir_id': df_to_check[borrowdir_col] if borrowdir_col in df_to_check.columns else ''
            })
            
            # Save to Excel
            hathi_df.to_excel(temp_file, index=False)
            print(f"✅ Prepared data saved to: {temp_file}")
            
            # Initialize scanner with conservative rate limiting
            scanner = HathiTrustFullScanner(rate_limit_delay=0.3, max_workers=3)
            
            # Run the scan
            print(f"\nStarting HathiTrust scan for {description} holdings...")
            print("This may take several minutes depending on the number of records...")
            
            # ADDED: Create output filename based on dataset type
            output_suffix = f"_{df_name}" if df_name else ""
            scanner.scan_full_file(temp_file, batch_size=50, output_suffix=output_suffix)
            
            # Results are automatically saved by the scanner
            print(f"\n✅ HathiTrust check complete for {description} holdings!")
            
            # Clean up temporary file
            if os.path.exists(temp_file):
                os.remove(temp_file)
            
            # If this was a sample, extrapolate results
            if 'sample_info' in locals() and df_name == 'ml_filtered_sample':
                print(f"\n📊 Extrapolating results to full ML-filtered dataset:")
                print(f"   Sample checked: {sample_info['sample_size']:,} records")
                print(f"   Full dataset: {sample_info['full_dataset_size']:,} records")
                print(f"   Results in 'hathitrust/reports' can be extrapolated by factor of {1/sample_info['sample_rate']:.1f}")
        
        print("\n📁 Check the 'hathitrust/reports' directory for detailed results")
        if 'sample_info' in locals():
            print("   📌 Note: Results are from a representative sample of the ~1M ML-filtered dataset")
            print("   Multiply findings by the extrapolation factor for full dataset estimates")
        
except ImportError:
    print("❌ Could not import HathiTrust scanner")
    print("Please ensure hathitrust_availability_checker_excel.py is in the hathitrust/ directory")
except Exception as e:
    print(f"❌ Error during HathiTrust check: {str(e)}")
    
    # Clean up on error - check for any temp files
    if 'temp_file' in locals() and os.path.exists(temp_file):
        os.remove(temp_file)
    
    # ADDED: Clean up any other temp files that might exist
    for temp_pattern in ['temp_hathitrust_input_df_penn_only.xlsx', 'temp_hathitrust_input_df_indeterminate.xlsx', 'temp_hathitrust_input_ml_filtered_1m.xlsx', 'temp_hathitrust_input_ml_filtered_sample.xlsx']:
        temp_path = os.path.join('pod-processing-outputs', temp_pattern)
        if os.path.exists(temp_path):
            os.remove(temp_path)